# 新三板企业数据详情爬取

# 注：首先去网站http://www.neeq.com.cn下载下来公司code，类似20190111161848_c27jcvswfr.xlsx

In [ ]:
import xlrd
import requests
import json
import pymysql
from datetime import datetime
def readexcel(path):
    workbook = xlrd.open_workbook(path)
    sheets = workbook.sheet_names()
    #worksheet = workbook.sheet_by_name(sheets[0])
   # worksheet = workbook.sheets()[0]
    worksheet = workbook.sheet_by_index(0)
    data=[]
    for i in range(0, worksheet.ncols):
        row = worksheet.row(i)
        data.append(worksheet.col_values(i))
    return data
data=readexcel('C:\\Users\\Administrator\\Desktop\\20190111161848_c27jcvswfr.xlsx')
#定义一个getjson函数用来解析返回的数据
def getjson(c):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }
    url='http://www.neeq.com.cn/nqhqController/detailCompany.do?callback=jQuery18303499613453215563_1542331950561&zqdm=%s&_=1542331957847'%str(int(c))
    response=requests.get(url=url,headers=headers)
    html=response.text
    html=html.lstrip("'jQuery18303499613453215563_1542331950561(").rstrip(")'")
    decodejson=eval(html)
    return decodejson

#连接数据库、获得游标，获取数据并插入到数据库中
#在获取数据时使用get()方法比较好，避免造成无相关数据时程序的中断
conn=pymysql.connect(host='localhost',user='root',password='123456',db='baidudata',charset='utf8')
cur=conn.cursor()
cur.execute("DROP TABLE IF EXISTS xinsanbandetail")
aql1='''CREATE TABLE xinsanbandetail(
                    ID int primary key auto_increment not null,
                    code VARCHAR(50) not null,
                    name VARCHAR(50) not null,
                    industry VARCHAR(50) not null,
                    address VARCHAR(200)  null,
                    postcode VARCHAR(200) not null,
                    broker VARCHAR(50) not null,
                    area VARCHAR(50) not null,
                    TM VARCHAR(50) not null);'''
try:
    cur.execute(aql1)
except Exception as e:
            print(e)
for c in data[0][4:]:
    decodejson=getjson(c)
    #print(decodejson)
    if decodejson.get('baseinfo'):
        result=decodejson.get('baseinfo')
        code=result.get('code')
        name=result.get('name')
        industry =result.get('industry')
        address=result.get('address')
        postcode=result.get('postcode')
        broker=result.get('broker')
        area=result.get('area')
        TM=result.get('listingDate')
        sql="""INSERT INTO xinsanbandetail
(code,name,industry,address,postcode,broker,area,TM)
VALUES ('%s','%s','%s','%s','%s','%s','%s','%s');"""%(code,name,industry,address,postcode,broker,area,TM)
        #print(sql)
        cur.execute(sql)
        conn.commit()
cur.close()
conn.close()
